# fastText und glove

Als Embeddings kennst du jetzt *word2vec*. Es gibt allerdings noch Alternativen dazu.

Von Facebook wurde [fastText](https://fasttext.cc/) "erfunden". *fastText* geht einen etwas anderen Weg als *word2vec* und nutzt auch noch Buchstaben-n-Gramme zum Training. Dadurch werden auch syntaktisch ähnliche Wörter gefunden und es ist ziemlich stabil gegen Tippfehler, weswegen es sich auch zur Landesspracherkennung eignet (die hast du schon kennengelernt). *fastText* bietet dir außerdem bereits sehr viele vortrainierte Modell zum Download an und ist damit ein sehr komplettes Paket.

Parallel zu *word2vec* bei Google wurde in Stanford [gloVe]() entwickelt. *gloVe* wird über eine Co-Occurrence-Matrix trainiert und produziert daher signifikant andere Resultate. Leider lässt sich *gloVe* nicht in Python trainieren, so sind wir auf ein CLI-Programm ausgewichen und stellen die Embeddings bereit.

## Daten einladen

Wie gewohnt lädst du die linguistisch analysierten Daten ein:

In [ ]:
!pip install "gensim>=4.0.0"

In [ ]:
import sys, os
ON_COLAB = 'google.colab' in sys.modules

if ON_COLAB:
    os.system("test -f heise-articles-2020.db || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.db.gz && gunzip heise-articles-2020.db.gz")
    newsticker_db = 'heise-articles-2020.db'
else:
    newsticker_db = '../99_Common/heise-articles-2020.db'

In [ ]:
import sqlite3 
import pandas as pd

sql = sqlite3.connect(newsticker_db)
df = pd.read_sql("SELECT * FROM nlp_articles WHERE datePublished<'2021-01-01' ORDER BY datePublished", 
                 sql, index_col="id", parse_dates=["datePublished"])

## fastText Embeddings trainieren

Du kennst schon die Datenvorbereitung für `gensim`:

In [ ]:
import regex as re
from spacy.lang.de.stop_words import STOP_WORDS as stop_words
gensim_words = [[w for w in re.split(r'[\\|\\#]', doc.lower()) if w not in stop_words]
                    for doc in df["nav"]]

`FastText` wird  genauso trainiert wie `Word2Vec`:

In [ ]:
from gensim.models import FastText
ft = FastText(gensim_words, min_count=5)
ft.wv.save_word2vec_format("heise-articles-2020.ft")

Und auch das API ist identisch. Hier siehst du bereits die Effekter der Buchstaben-n-Gramme: `jva` würde bei `Word2Vec` keine hohe Ähnlichkeit haben, ebenso nicht die ganzen Begriffe, die `javascript` zuzuordnen sind:

In [ ]:
ft.wv.most_similar(positive=["java"], topn=10)

Bei `microsoft` und `corona` ist der Effekt noch ausgeprägter:

In [ ]:
ft.wv.most_similar(positive=["microsoft"], topn=10)

In [ ]:
ft.wv.most_similar(positive=["corona"], topn=10)

Die Analogieschlüsse funktionieren aber ähnlich gut:

In [ ]:
pd.DataFrame(ft.wv.most_similar(positive=["apple", "android"], 
                                   negative=["google"],  topn=10))

In [ ]:
pd.DataFrame(ft.wv.most_similar(positive=["microsoft", "android"], 
                                   negative=["google"],  topn=10))

## GloVe Embeddings

Zunächst musst du dazu das File `heise-articles-nav-2020.txt` erzeugen:

In [ ]:
open("heise-articles-nav-2020.txt", "w").write("\n".join([n.lower() 
                                                          for n  in df["nav"].str.replace(r'\#|\|', ' ').values]))

Leider gibt es in Python keine direkte Möglichkeit, GloVe-Embeddings zu trainieren. Wir haben sie daher mit folgendem Skript vortrainiert:

```sh
VOCAB_FILE=vocab.txt
COOCCURRENCE_FILE=cooccurrence.bin
COOCCURRENCE_SHUF_FILE=cooccurrence.shuf.bin
BUILDDIR=build
SAVE_FILE=heise-articles-2020.glove
VERBOSE=2
MEMORY=4.0
VOCAB_MIN_COUNT=5
VECTOR_SIZE=50
MAX_ITER=15
WINDOW_SIZE=15
BINARY=2
NUM_THREADS=8
X_MAX=10
CORPUS=heise-articles-nav-2020.txt
BUILDDIR=~/src/GloVe-1.2/build/
$BUILDDIR/vocab_count -min-count $VOCAB_MIN_COUNT -verbose $VERBOSE < $CORPUS > $VOCAB_FILE
$BUILDDIR/cooccur -memory $MEMORY -vocab-file $VOCAB_FILE -verbose $VERBOSE -window-size $WINDOW_SIZE <$CORPUS > $COOCCURRENCE_FILE
$BUILDDIR/shuffle -memory $MEMORY -verbose $VERBOSE < $COOCCURRENCE_FILE > $COOCCURRENCE_SHUF_FILE
$BUILDDIR/glove -save-file $SAVE_FILE -threads $NUM_THREADS -input-file $COOCCURRENCE_SHUF_FILE -x-max $X_MAX -iter 100 -vector-size $VECTOR_SIZE -binary $BINARY -vocab-file $VOCAB_FILE -verbose $VERBOSE
```

Das resultierende File kannst du direkt in `gensim` einladen:

In [ ]:
if ON_COLAB:
    os.system("heise-articles-2020.glove.txt || wget  https://datanizing.com/heiseacademy/nlp-course/blob/main/99_Common/heise-articles-2020.glove.txt.gz && gunzip heise-articles-2020.glove.txt.gz")
    glove_file = 'heise-articles-2020.glove.txt'
else:
    glove_file = '../99_Common/heise-articles-2020.glove.txt'

In [ ]:
from gensim.models import KeyedVectors
glove = KeyedVectors.load_word2vec_format(glove_file, no_header=True)

Am besten vergleichst du die Ergebnisse mit denen von `FastText` oben:

In [ ]:
glove.most_similar(positive=["java"], topn=10)

Das ist deutlich konzeptioneller!

Wie sieht es bei `Microsoft` aus?

In [ ]:
glove.most_similar(positive=["microsoft"], topn=10)

Einige relativ allgemeine Wörter tauchen plötzlich auf. Aber sonst sieht das Ergebnis prima aus. Ebenso ist es bei `Corona`:

In [ ]:
glove.most_similar(positive=["corona"], topn=10)

Die Analogieschlüsse funktionieren noch besser als bei `Word2Vec`:

In [ ]:
pd.DataFrame(glove.most_similar(positive=["apple", "android"], 
                                   negative=["google"],  topn=10))

In [ ]:
pd.DataFrame(glove.most_similar(positive=["microsoft", "android"], 
                                   negative=["google"],  topn=10))

## FastText und Glove nach Anwendungsfall auswählen

*FastText* konzentriert sich auf neben der semantischen Ähnlichkeit besonders auf die Klassifikation *unscharfer Daten* und die Identifikation von Wörtern, die ähnlich geschrieben werden. *FastText* ist perfekt in `gensim` eingebunden und kann genau wie `Word2Vec` verwendet werden.

*GloVe* hingegen arbeitet deutlich konzeptioneller und ist leider nicht direkt als Python-Bibliothek vorhanden. Dadurch ist wahrscheinlich auch seine geringere Popularität zu erklären, die Ergebnisse sind nämlich (abgesehen von einigen generischen Wörtern) sehr gut.